---
title: "Engineering a computational surface modeling toolbox in Julia"
# subtitle: "An introduction to FreeBird.jl"
author: "Ray M. Yang"
# date: "Nov 1, 2024"
institute: "Washington University in St. Louis"
location: "PChem Group Meeting"
jupyter: julia-1.11
format: #revealjs
  revealjs:
    # fontsize: 2em
    # theme: dark
    slides-url: https://samforeman.me/talks/llms-at-scale/slides.html
    template-partials:
      - ./title-slide.html 
      # - ./title-fancy/title-slide.html
      # - ./title_slide_template.html
      # - ../../title-slide.html
    title-slide-attributes:
      data-background-iframe: https://emilhvitfeldt.github.io/quarto-iframe-examples/colored-particles/index.html
      data-background-size: contain
      data-background-color: white
      background-color: white
    mermaid:
      theme: neutral
    scrollable: true
    background-color: white
    # output-file: "slides.html"
    navigation-mode: linear
    # title-block-style: none 
    slide-number: c
    # title-slide-style: default
    chalkboard:
      # buttons: false
      chalkboard: true
    auto-animate: true
    reference-location: section
    touch: true
    pause: false
    footnotes-hover: true
    citations-hover: true
    preview-links: true
    controls-tutorial: true
    controls: false
    logo: WashU.png
    history: false
    highlight-style: "atom-one"
    # center: true
    default-image-extension: svg
    code-overflow: scroll
    html-math-method: katex
    fig-align: center
    css:
      # - css/default.css
      - css/custom.css
    # theme:
    #   # - light:
    #   - white
    #   - ../../css/title-slide-template.scss
    #   - ../../css/reveal/reveal.scss
    #   - ../../css/common.scss
    #   - ../../css/light.scss
    #   - ../../css/syntax-light.scss
    #   - ../../css/callout-cards.scss
      # - dark:
      #   - black
      #   - ./title-fancy/title-slide-template.scss
      #   - ../../css/reveal/reveal.scss
      #   - ../../css/common.scss
      #   - ../../css/dark.scss
      #   - ../../css/syntax-dark.scss
      #   - ../../css/callout-cards.scss
    # theme: [title-fancy/title-slide-template.scss]
    callout-style: simple
    # css: [css/default.css, css/callouts.css]
---




## Overview


1. **Introduction to `FreeBird.jl`**  
    Design, functionality, and extensibility


2. **Introduction to Research Software Engineering (RSE)**  
    What is RSE, and how does it fit in Chemistry


## What is `FreeBird.jl`

- It's a computational package
- It's developed by the Wexler Group
- It does surface *free* energy calculations via *nested* sampling
- It's written in [Julia](https://julialang.org/), a modern, high-level programming language


## What is [Julia](https://julialang.org/)?

<img align="left" src="https://julialang.org/assets/infra/logo.svg" height="80" style="padding-right: 20px;"/>  is a high-level, high-performance, dynamic programming language for technical computing. It's fast, easy to learn, and has a syntax that is familiar to users of other technical computing environments.

- Version 1.0 was released in 2018
- Developed by mathematician Alan Edelman *et al.* at MIT
- It's designed for high-performance numerical analysis and computational science
- It's free and open source



## `FreeBird.jl` design

- It's modular
- It's extensible
- It's user-friendly
- It's well-documented
- It's open-source


## `FreeBird.jl` functionality

- Systems: atomistic and lattice models
- Energy calculator: Lennard-Jones and lattice Hamiltonians
- Methods: nested sampling, Metropolis Monte Carlo, exact enumeration, Wang-Landau sampling


## Nested sampling



## Atomistic walkers


In [ ]:
#| output: none
#| eval: true
if isfile("output_df.csv")
  run(`rm output_df.csv output.traj.extxyz`) # delete previous output
end

In [ ]:
#| echo: true
#| eval: true
using FreeBird
walkers = AtomWalker.(generate_initial_configs(120, 562.5, 6))
lj = LJParameters(epsilon=0.1, sigma=2.5)

## Atomistic walkers

In [ ]:
#| echo: true
walkers[1]

## Atomistic walkers

In [ ]:
#| echo: true
walkers[1].configuration

## Construct the liveset


In [ ]:
#| echo: true
ls = LJAtomWalkers(walkers, lj)

## Run nested sampling


In [ ]:
#| echo: true
#| eval: true
mc = MCRandomWalkClone()
ns_params = NestedSamplingParameters(200, 0.1, 0.01, 1e-5, 1.0, 0, 200)
save = SaveEveryN(n_traj=10, n_snap=10_000)
energies, liveset, _ = nested_sampling_loop!(ls, ns_params, 10_000, mc, save)

## Nested sampling results


In [ ]:
#| echo: true
liveset.walkers[1].configuration

## Nested sampling results


In [ ]:
using Plots
plotly()
plot(energies.iter[1:end], energies.emax[1:end], ylabel="Energy", xlabel="Iteration", label="")

## Nested sampling results


In [ ]:
plot(energies.iter[100:end], energies.emax[100:end], ylabel="Energy", xlabel="Iteration", label="")

## Run nested sampling

Nested sampling in 7 lines of code!


In [ ]:
#| echo: true
#| eval: false
ls = LJAtomWalkers(AtomWalker.(generate_initial_configs(120, 562.5, 6)), LJParameters())
mc = MCRandomWalkClone()
ns_params = NestedSamplingParameters(200, 0.1, 0.01, 1e-5, 1.0, 0, 200)
save = SaveEveryN(n_traj=10, n_snap=10_000)
energies, liveset, _ = nested_sampling_loop!(ls, ns_params, 10_000, mc, save)

Or, in a single line if you really want to!

In [ ]:
#| echo: true
#| eval: false
energies, liveset, _ = nested_sampling_loop!(LJAtomWalkers(AtomWalker.(generate_initial_configs(100, 100.0, 6)), LJParameters()), NestedSamplingParameters(200, 0.1, 0.01, 1e-5, 1.0, 0, 200), 10_000, MCRandomWalkClone(), SaveEveryN(n_traj=10, n_snap=10_000))

## Accessing the documentation

```julia
help?> nested_sampling_loop!
search: nested_sampling_loop! nested_sampling_step!

  nested_sampling_loop!(liveset::AtomWalkers, ns_params::NestedSamplingParameters, n_steps::Int64, mc_routine::MCRoutine; args...)

  Perform a nested sampling loop for a given number of steps.

  Arguments
  ≡≡≡≡≡≡≡≡≡

    •  liveset::AtomWalkers: The initial set of walkers.

    •  ns_params::NestedSamplingParameters: The parameters for nested sampling.

    •  n_steps::Int64: The number of steps to perform.

    •  mc_routine::MCRoutine: The Monte Carlo routine to use.

  Returns
  ≡≡≡≡≡≡≡

    •  df: A DataFrame containing the iteration number and maximum energy for each step.

    •  liveset: The updated set of walkers.

    •  ns_params: The updated nested sampling parameters.

  ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

  nested_sampling_loop!(liveset::LatticeGasWalkers, ns_params::LatticeNestedSamplingParameters, n_steps::Int64, mc_routine::MCRoutine; args...)

  Perform a nested sampling loop on a lattice gas system for a given number of steps.

  Arguments
  ≡≡≡≡≡≡≡≡≡

    •  liveset::LatticeGasWalkers: The initial set of walkers.

    •  ns_params::LatticeNestedSamplingParameters: The parameters for nested sampling.

    •  n_steps::Int64: The number of steps to perform.

    •  mc_routine::MCRoutine: The Monte Carlo routine to use.

  Keyword Arguments
  ≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡

    •  args...: Additional arguments.

  Returns
  ≡≡≡≡≡≡≡

    •  df: A DataFrame containing the iteration number and maximum energy for each step.

    •  liveset: The updated set of walkers.

    •  ns_params: The updated nested sampling parameters.

  ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

  nested_sampling_loop!(liveset::AtomWalkers, n_steps::Int64, mc_routine::MixedMCRoutine, save_strategy::DataSavingStrategy)

  Perform a nested sampling loop for a given number of steps.

  Arguments
  ≡≡≡≡≡≡≡≡≡

    •  liveset::AtomWalkers: The initial set of walkers.

    •  n_steps::Int64: The number of steps to perform.

    •  mc_routine::MixedMCRoutine: The mixed Monte Carlo routine to use.

    •  save_strategy::DataSavingStrategy: The strategy for saving data.

  Returns
  ≡≡≡≡≡≡≡

    •  df::DataFrame: The data frame containing the iteration number and maximum energy for each step.

    •  liveset::AtomWalkers: The updated set of walkers.

    •  mc_routine.ns_params_main: The updated nested sampling parameters for the main routine.
```

## Accessing the documentation

```julia
help?> NestedSamplingParameters
search: NestedSamplingParameters LatticeNestedSamplingParameters nested_sampling_step!

  mutable struct NestedSamplingParameters <: SamplingParameters

  The NestedSamplingParameters struct represents the parameters used in the nested sampling scheme.

  Fields
  ≡≡≡≡≡≡

    •  mc_steps::Int64: The number of total Monte Carlo moves to perform.

    •  initial_step_size::Float64: The initial step size, which is the fallback step size if MC routine fails to accept a move.

    •  step_size::Float64: The on-the-fly step size used in the sampling process.

    •  step_size_lo::Float64: The lower bound of the step size.

    •  step_size_up::Float64: The upper bound of the step size.

    •  fail_count::Int64: The number of failed MC moves in a row.

    •  allowed_fail_count::Int64: The maximum number of failed MC moves allowed before resetting the step size.
```

## Finding phase transitions


In [ ]:
#| echo: true
#| eval: true
# Calculate the ω factors
ωi = ωᵢ(energies.iter, 100)
# Shift the energies to be greater than or equal to zero
Ei = energies.emax .- minimum(energies.emax)
# Specify the temperatures that we are interested in
Ts = collect(1:0.1:10000)
# Define the Boltzmann constant
kb = 8.617333262e-5 # eV/K
# Calculate the inverse temperatures
β = 1 ./(kb.*Ts)
# Define the degrees of freedom, which is 3×6 for the 6-particle system
dof = 18
# Calculate the partition functions for each temperature
Zs = [partition_function(b, ωi, Ei) for b in β]
# Calculate the internal energies for each temperature
U = [internal_energy(b, ωi, Ei) for b in β]
# Calculate the heat capacities as a function of temperature
cvs = cv(energies, β, dof, 100)

## Finding surface phase transitions


In [ ]:
#| echo: false
#| eval: true
plot(Ts, cvs./kb, label="FreeBird", xlabel="T(K)", ylabel="Heat Capacity (Kb)", title="6-particle Heat Capacity", legend=:topleft)

## Surface stuff


In [ ]:
#| output: none
if isfile("output_df.csv")
  run(`rm output_df.csv output.traj.extxyz`) # delete previous output
end

In [ ]:
#| echo: true
using AtomsBase
configs =  read_configs("slab.extxyz",pbc="TTF")
configs = FastSystem.(configs)
slabs = AtomWalker{2}.(configs, list_num_par=[80,6],frozen=[1,0])

## Surface stuff

In [ ]:
#| echo: true
slabs[1].configuration

## Surface stuff
<!-- ```{julia}
#| echo: true
#| eval: false
surf_lj = LJParameters(epsilon=0.1, sigma=2.5, cutoff=4.0)
surf_ls = LJAtomWalkers(slabs, surf_lj)
mc = MCRandomWalkClone()
surf_ns_params = NestedSamplingParameters(500, 0.1, 0.01, 1e-5, 1.0, 0, 200)
save = SaveEveryN(n_traj=10, n_snap=20_000)
surf_energies, liveset, _ = nested_sampling_loop!(surf_ls, surf_ns_params, 50_000, mc, save)
``` -->

## Surface results

<!-- ```{julia}
#| echo: true
#| eval: false
surf_ls.walkers[1].configuration
``` -->



## What's next

- Publish! There are some journals that are specifically designed for software papers

::: {style="text-align: center; margin-top: 1em"}
[Journal of Open Source Software](https://joss.theoj.org/){preview-link="true" style="text-align: center"}
:::

Or, traditional journals like: 

::: {style="text-align: center; margin-top: 1em"}
[The Journal of Chemical Physics](https://pubs.aip.org/aip/jcp/article/159/16/164101/2918010/ACEpotentials-jl-A-Julia-implementation-of-the){preview-link="true" style="text-align: center"}
:::


## Take-home message
- Version control is a powerful tool
- It's a good practice for code development
- It's essential for collaborative projects
- It has great importance in scientific reproducibility

Start using it, now!